In [ ]:
from datetime import datetime
import time
import win32com.client
import os
import pyautogui
from openpyxl import load_workbook,Workbook

In [ ]:
def get_session():
    try:
        # Abrir instancia en SAP
        SapGuiAuto = win32com.client.GetObject("SAPGUI")
        app = SapGuiAuto.GetScriptingEngine
        connection = app.Children(0)
        session = connection.Children(0)
        return session
    except Exception as e:
        print("Error GET_SESSION:", str(e))

def enter_tx(tx):
    global session
    try:
        session.findById("wnd[0]/tbar[0]/okcd").text = f"/n{tx}"
        session.findById("wnd[0]").sendVKey(0)
    except Exception as e:
        print("Error ENTER_TX:", str(e))

In [ ]:
# Cargar archivo de asientos
archivo = r"C:\Users\Ext_rgutierrezc\OneDrive - Grupo Alicorp\Escritorio\Proyecto automatización SAP\archivo.xlsx"
wb = load_workbook(archivo)
ws=wb['DATA']
sociedad=770

# Crear un nuevo libro de Excel y una hoja
wb = Workbook()
ws_output = wb.active
ws_output.title = "Datos SAP"

# Agregar encabezados (si es necesario)
ws_output.append(["Nro Documento", "Cuenta", "Nombre Acreedor", "Nro Acreedor", "USD Factura", "PEN Factura", 
                  "Nro Factura", "Fecha Factura", "Nombre Banco", "Cuenta Banco", "USD Banco", 
                  "PEN Banco", "Nro Banco", "Fecha Banco", "Error"])


In [ ]:
# Obtener la sesion de SAP
session = get_session()

for i, fila in enumerate(ws.iter_rows(min_row=2, max_row=ws.max_row, values_only=True), start=2):
    # Solo filas con datos
    nro_documento = fila[0]
    cuenta = fila[1]
    if not nro_documento:
        continue

    try:
        ejercicio = fila[6].strftime("%Y")
        
        # Ir a la TX 
        enter_tx("FB03")
        session.findById("wnd[0]").maximize()

        # Nro Documento
        session.findById("wnd[0]/usr/txtRF05L-BELNR").text = nro_documento
        # Sociedad
        session.findById("wnd[0]/usr/ctxtRF05L-BUKRS").text = sociedad
        # Ejercicio
        session.findById("wnd[0]/usr/txtRF05L-GJAHR").text = ejercicio
        # Referencia
        session.findById("wnd[0]/usr/txtRF05L-XBLNR").text = ""
        # Ejecutar
        session.findById("wnd[0]").sendVKey(0)

        # Filtrar Cuenta
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").selectColumn("KTONR")
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").pressToolbarButton("&MB_FILTER")
        session.findById("wnd[1]/usr/ssub%_SUBSCREEN_FREESEL:SAPLSSEL:1105/ctxt%%DYN001-LOW").text = cuenta
        session.findById("wnd[1]/tbar[0]/btn[0]").press()

        # Doble click primera fila
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").currentCellColumn = "KTONR"
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").selectedRows = "0"
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").doubleClickCurrentCell()

        # Extraer la información
        nombre_acreedor = session.findById("wnd[0]/usr/txtLFA1-NAME1").text
        nro_acreedor = session.findById("wnd[0]/usr/ctxtLFA1-LIFNR").text
        USD_factura = session.findById("wnd[0]/usr/txtBSEG-WRBTR").text
        PEN_factura = session.findById("wnd[0]/usr/txtBSEG-DMBTR").text
        nro_factura = session.findById("wnd[0]/usr/txtBSEG-AUGBL").text
        fecha_factura = session.findById("wnd[0]/usr/ctxtBSEG-AUGDT").text

        # Doble click en Nro compensación
        session.findById("wnd[0]/usr/txtBSEG-AUGBL").setFocus()
        session.findById("wnd[0]/usr/txtBSEG-AUGBL").caretPosition = 8
        session.findById("wnd[0]").sendVKey(2)

        #Filtrar por el acreedor (nuevamente)
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").selectColumn("KTONR")
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").pressToolbarButton("&MB_FILTER")
        session.findById("wnd[1]/usr/ssub%_SUBSCREEN_FREESEL:SAPLSSEL:1105/ctxt%%DYN001-LOW").text = cuenta
        session.findById("wnd[1]/tbar[0]/btn[0]").press()

        # Doble click en la cuenta (nuevamente)
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").currentCellColumn = "KTONR"
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").selectedRows = "0"
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").doubleClickCurrentCell()

        # Doble click en la compensacion (nuevamente)
        session.findById("wnd[0]/usr/txtBSEG-AUGBL").setFocus()
        session.findById("wnd[0]/usr/txtBSEG-AUGBL").caretPosition = 8
        session.findById("wnd[0]").sendVKey(2)

         # Filtrar por 10* (nuevamente)
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").selectColumn("KTONR")
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").pressToolbarButton("&MB_FILTER")
        session.findById("wnd[1]/usr/ssub%_SUBSCREEN_FREESEL:SAPLSSEL:1105/ctxt%%DYN001-LOW").text = "10*"
        session.findById("wnd[1]/tbar[0]/btn[0]").press()

        # Doble Click en la primera linea (banco) 
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").currentCellColumn = "KTONR"
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").selectedRows = "0"
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").doubleClickCurrentCell()

        # Extraer DATOS
        cuenta_banco = session.findById("wnd[0]/usr/ctxtBSEG-HKONT").text
        nombre_banco = session.findById("wnd[0]/usr/txtSKAT-TXT50").text
        USD_banco = session.findById("wnd[0]/usr/txtBSEG-WRBTR").text
        PEN_banco = session.findById("wnd[0]/usr/txtBSEG-DMBTR").text
        nro_banco = session.findById("wnd[0]/usr/txtBSEG-AUGBL").text
        fecha_banco = session.findById("wnd[0]/usr/ctxtBSEG-AUGDT").text

        # Almacenar datos en Excel
        ws_output.append([nro_documento, cuenta, nombre_acreedor, nro_acreedor, USD_factura, PEN_factura, nro_factura, fecha_factura, nombre_banco, cuenta_banco, USD_banco, PEN_banco, nro_banco, fecha_banco, ""])

    except Exception as e:
        # En caso de error, almacenar el error en la columna final
        ws_output.append([nro_documento, cuenta, "", "", "", "", "", "", "", "", "", "", "", "", str(e)])

# Guardar el archivo Excel
wb.save("output_sap_data.xlsx")